In [75]:
import os
import sqlite3
from pathlib import Path
import csv
import utils
import psycopg2

In [76]:
chemin_movies = "data/movies.csv"
chemin_links = "data/links.csv"
chemin_ratings = "data/ratings.csv"
chemin_tags = "data/tags.csv"

In [77]:
def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5432):
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5432)
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    conn.set_session(autocommit=True)
    return conn

In [78]:
ma_base_donnees = "Movies2"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw')

In [79]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

psycopg2.extensions.connection

In [110]:
def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

    
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été créée avec succès")
    
def creer_colonne(conn, pgsql_creation_colonne):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_creation_colonne)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la colonne")
        print(e)
        return
    cursor.close()
    print("La colonne a été créée avec succès")

    
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

    
def lire_donnees(conn, sql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data

In [111]:
creer_table(conn, pgsql_creer_table)

La table a été créée avec succès


In [112]:
pgsql_creer_table = """
    CREATE TABLE IF NOT EXISTS movies (
    movieId INT UNIQUE PRIMARY KEY,
    title VARCHAR NOT NULL,
    genres VARCHAR NOT NULL);

    CREATE TABLE IF NOT EXISTS links (
    movieId INT PRIMARY KEY NOT NULL,
    imdbId VARCHAR,
    tmdbId VARCHAR,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));

    CREATE TABLE IF NOT EXISTS ratings (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    rating REAL NOT NULL,
    timestamp INT NOT NULL,
    PRIMARY KEY (userId, movieId),
    foreign key(movieId) REFERENCES movies(movieId));

    CREATE TABLE IF NOT EXISTS tags (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    tag VARCHAR NOT NULL,
    timestamp INT NOT NULL,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));
"""

In [113]:
pgsql_creer_colonne = """
    ALTER TABLE movies
    ADD year integer;
    
    ALTER TABLE movies
    ADD title_only VARCHAR;
"""

In [114]:
pgsql_inserer_movies = """
    INSERT INTO movies 
    (movieId, title, genres)
    VALUES (%(movieId)s, %(title)s, %(genres)s);
"""

pgsql_inserer_links = """
    INSERT INTO links 
    (movieId, imdbId, tmdbId)
    VALUES (%(movieId)s, %(imdbId)s, %(tmdbId)s);
"""

pgsql_inserer_ratings = """
    INSERT INTO ratings 
    (userId, movieId, rating, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(rating)s, %(timestamp)s);
"""
pgsql_inserer_tags = """
    INSERT INTO tags 
    (userId, movieId, tag, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(tag)s, %(timestamp)s);
"""

In [115]:
def lire_csv_dict(nom_fichier):
    data = []
    with open(nom_fichier, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

In [116]:
movies_dict = lire_csv_dict(chemin_movies)

In [117]:
tags_dict = lire_csv_dict(chemin_tags)

In [118]:
links_dict = lire_csv_dict(chemin_links)

In [119]:
ratings_dict = lire_csv_dict(chemin_ratings)

In [120]:
inserer_donnees(conn, pgsql_inserer_movies, movies_dict)

Erreur lors de l'insertion des données
ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « movies_pkey »
DETAIL:  La clé « (movieid)=(1) » existe déjà.



In [121]:
inserer_donnees(conn, pgsql_inserer_ratings, ratings_dict)

Erreur lors de l'insertion des données
ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « ratings_pkey »
DETAIL:  La clé « (userid, movieid)=(1, 1) » existe déjà.



In [122]:
inserer_donnees(conn, pgsql_inserer_tags, tags_dict)

Les données ont été insérées avec succès


In [123]:
inserer_donnees(conn, pgsql_inserer_links, links_dict)

Erreur lors de l'insertion des données
ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « links_pkey »
DETAIL:  La clé « (movieid)=(1) » existe déjà.



### Quels sont les 15 films les mieux notés ?

In [124]:
films_mieux_notes = """
    SELECT title, count(rating), avg(rating) \
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId \
    GROUP BY title \
    ORDER BY avg(rating) DESC \
    LIMIT 15
"""

In [125]:
lire_donnees(conn, films_mieux_notes)

Les données ont été lues avec succès


[('Mystery of the Third Planet, The (Tayna tretey planety) (1981)', 1, 5.0),
 ('Garden of Words, The (Koto no ha no niwa) (2013)', 1, 5.0),
 ('Brother (Brat) (1997)', 1, 5.0),
 ('Into the Woods (1991)', 1, 5.0),
 ('Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)',
  1,
  5.0),
 ('Meantime (1984)', 1, 5.0),
 ('Nasu: Summer in Andalusia (2003)', 1, 5.0),
 ('Holy Motors (2012)', 1, 5.0),
 ('Watching the Detectives (2007)', 1, 5.0),
 ('Eva (2011)', 1, 5.0),
 ('Story of Women (Affaire de femmes, Une) (1988)', 1, 5.0),
 ('Connections (1978)', 1, 5.0),
 ('Go for Zucker! (Alles auf Zucker!) (2004)', 1, 5.0),
 ('Girls About Town (1931)', 1, 5.0),
 ('What Men Talk About (2010)', 1, 5.0)]

### Quels sont les 15 films ayant reçu plus de 40 notes les mieux notés ?

In [126]:
films_mieux_notes_40 = """
    SELECT title, count(rating), cast(avg(rating) AS DECIMAL (19,2)) \
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId \
    GROUP BY title \
    HAVING count(rating) > 40 \
    ORDER BY avg(rating) DESC \
    LIMIT 15
"""

In [127]:
lire_donnees(conn, films_mieux_notes_40)

Les données ont été lues avec succès


[('Shawshank Redemption, The (1994)', 317, Decimal('4.43')),
 ('Lawrence of Arabia (1962)', 45, Decimal('4.30')),
 ('Godfather, The (1972)', 192, Decimal('4.29')),
 ('Fight Club (1999)', 218, Decimal('4.27')),
 ('Cool Hand Luke (1967)', 57, Decimal('4.27')),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  97,
  Decimal('4.27')),
 ('Rear Window (1954)', 84, Decimal('4.26')),
 ('Godfather: Part II, The (1974)', 129, Decimal('4.26')),
 ('Departed, The (2006)', 107, Decimal('4.25')),
 ('Goodfellas (1990)', 126, Decimal('4.25')),
 ('Casablanca (1942)', 100, Decimal('4.24')),
 ('Dark Knight, The (2008)', 149, Decimal('4.24')),
 ('Usual Suspects, The (1995)', 204, Decimal('4.24')),
 ('Princess Bride, The (1987)', 142, Decimal('4.23')),
 ('Star Wars: Episode IV - A New Hope (1977)', 251, Decimal('4.23'))]

### Quels sont les utilisateurs les plus prolifiques ?

In [128]:
sql_moyenne_user = """
    select userId, count(rating), cast(avg(rating) AS DECIMAL (19,2))
    from ratings r 
    group by userId 
    having count(rating) > 1000
    order by count(rating) desc
"""

In [129]:
lire_donnees(conn, sql_moyenne_user)

Les données ont été lues avec succès


[(414, 2698, Decimal('3.39')),
 (599, 2478, Decimal('2.64')),
 (474, 2108, Decimal('3.40')),
 (448, 1864, Decimal('2.85')),
 (274, 1346, Decimal('3.24')),
 (610, 1302, Decimal('3.69')),
 (68, 1260, Decimal('3.23')),
 (380, 1218, Decimal('3.67')),
 (606, 1115, Decimal('3.66')),
 (288, 1055, Decimal('3.15')),
 (249, 1046, Decimal('3.70')),
 (387, 1027, Decimal('3.26'))]

### Quels sont les 10 films recevant le plus de tags ?

In [130]:
films_plus_tags = """
SELECT title, COUNT(*)
    FROM tags INNER JOIN movies on tags.movieId=movies.movieId
    GROUP BY title
    ORDER BY COUNT(*) DESC
    LIMIT 15
"""

In [131]:
lire_donnees(conn, films_plus_tags)

Les données ont été lues avec succès


[('Pulp Fiction (1994)', 724),
 ('Fight Club (1999)', 216),
 ('2001: A Space Odyssey (1968)', 164),
 ('Léon: The Professional (a.k.a. The Professional) (Léon) (1994)', 140),
 ('Eternal Sunshine of the Spotless Mind (2004)', 136),
 ('Big Lebowski, The (1998)', 128),
 ('Donnie Darko (2001)', 116),
 ('Inception (2010)', 104),
 ('Star Wars: Episode IV - A New Hope (1977)', 104),
 ('Suicide Squad (2016)', 76),
 ('In the Mood For Love (Fa yeung nin wa) (2000)', 72),
 ('Avatar (2009)', 72),
 ('Pi (1998)', 68),
 ('Eraserhead (1977)', 68),
 ('Avengers: Infinity War - Part I (2018)', 60)]

### Quels sont les 25 tags les plus utilisés ? Avec quelle fréquence ?

In [132]:
sql_frequence_tags = """
select tag, count(*)
from tags t2 
group by tag
order by count(*) desc
limit 25
"""

In [133]:
lire_donnees(conn, sql_frequence_tags)

Les données ont été lues avec succès


[('In Netflix queue', 524),
 ('atmospheric', 144),
 ('thought-provoking', 96),
 ('superhero', 96),
 ('surreal', 92),
 ('funny', 92),
 ('Disney', 92),
 ('religion', 88),
 ('quirky', 84),
 ('psychology', 84),
 ('sci-fi', 84),
 ('dark comedy', 84),
 ('suspense', 80),
 ('visually appealing', 76),
 ('twist ending', 76),
 ('crime', 76),
 ('politics', 72),
 ('mental illness', 64),
 ('music', 64),
 ('time travel', 64),
 ('aliens', 60),
 ('dark', 60),
 ('comedy', 60),
 ('dreamlike', 56),
 ('space', 56)]

In [134]:
creer_colonne(conn, pgsql_creer_colonne)

Erreur lors de la création de la colonne
ERREUR:  la colonne « year » de la relation « movies » existe déjà

